In [1]:
import os
import gradio as gr
from gradio.components import HTML
import torch
from PIL.ImageOps import scale
from PIL import Image
from transformers import BertTokenizer, BertForSequenceClassification
import json
from zhipuai import ZhipuAI

C:\Users\10521\.conda\envs\Pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label_num_to_text = {0: ('Analgesics', 'Germany'), 1: ('Analgesics', 'Poland'), 2: ('Furniture', 'Germany'), 3: ('Antibiotics', 'Poland'), 4: ('Antibiotics', 'Germany'), 5: ('Antimalarial', 'Poland'), 6: ('Antimalarial', 'Germany'), 7: ('Antipiretics', 'Poland'), 8: ('Antipiretics', 'Germany'), 9: ('Antiseptics', 'Poland'), 10: ('Antiseptics', 'Germany'), 11: ('Mood Stabilizers', 'Poland'), 12: ('Mood Stabilizers', 'Germany')}


In [ ]:
# Load the BERT model and tokenizer
# model_path = '../Classification/results/checkpoint-1299'
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained(model_path)
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)

In [26]:
# Initialize ZhipuAI client
client = ZhipuAI(api_key="04c2e3e77614447f88083749a02e91c7.agrCgPanJ3kl4zg6")

In [20]:
# Function to classify pharmacy
def classify_commodity(commodity_name):
    # inputs = tokenizer(commodity_name, return_tensors="pt").to(device)
    # outputs = model(**inputs)
    # preds = outputs.logits.argmax(-1).item()
    # category = label_num_to_text[preds][0] + "-" + label_num_to_text[preds][1]
    # print(category)
    return "Analgesics-Germany"

In [32]:
# Function to generate sales suggestions
def generate_suggestions(messages, commodity_name, category):

    analysis_file = f'data_analysis/analysis_result/Pharm-data/analysis_result_{category}.json'
    with open(analysis_file, 'r') as f:
        analysis_result = json.load(f)


    known_facts = {
        "product": category,
        "top_10_distributor_performance": analysis_result["top_10_distributor_performance"],
        "team_performance": analysis_result["team_performance"],
        "sales_rep_distribution": analysis_result["sales_rep_distribution"],
        "top_10_cities_in_Germany_by_sales_of_Analgesics": analysis_result["top_10_cities_in_Germany_by_sales_of_Analgesics"],
        "top_10_customers_by_sales": analysis_result["top_10_customers_by_sales"],
        "top_10_products_by_sales": analysis_result["top_10_products_by_sales"],
        # "maximum_sales_record": analysis_result["sales_max"],
        # "25th_percentile_sales": analysis_result["sales_25th_percentile"],
        # "median_sales_amount": analysis_result["sales_50th_percentile"],
        # "75th_percentile_sales": analysis_result["sales_75th_percentile"],
        
    }
    # top_10_cities_image = Image.open(f'../data_analysis/analysis_result_{category}/top_10_cities_distribution_{category}.png')
    # top_10_states_image = Image.open(f'../data_analysis/analysis_result_{category}/top_10_states_distribution_{category}.png')
    # seasonal_analysis_image = Image.open(f'../data_analysis/analysis_result_{category}/Seasonal_Analysis_{category}.png')


    response = client.chat.completions.create(
        model="glm-4-flash-250414",
        messages=[
            {"role": "user", "content": f"As a pharmacy product analyst to help the pharmacy store, please analyze the sales for the drug {category}, with less than 300 words."},
            {"role": "assistant", "content": "Of course, I can help you with that. Could you provide me with some information about the product?"},
            {"role": "user", "content": f"Here are the facts: {known_facts}"}
        ],
    )
    sales_analysis = response.choices[0].message.content

    messages.append((commodity_name, f"The commodity '{commodity_name}' belongs to category '{category}'.\n\nAnalysis:\n{sales_analysis}"))
    return messages, known_facts

In [8]:
# Function to send a message to the chatbot to keep talking (remember using the history)
def send_message(chat_history, message):
    messages = []
    print(chat_history)
    for i in range(0, len(chat_history)):
        messages.append({
            "role": "user",
            "content": chat_history[i][0]
        })
        messages.append({
            "role": "assistant",
            "content": chat_history[i][1]
        })
    messages.append({
        "role": "user",
        "content": message
    })
    # using history to keep the conversation
    print(messages)
    response = client.chat.completions.create(
        model="glm-4-flash-250414",
        messages=messages,
    )
    messages.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })
    chat_history.append([message,response.choices[0].message.content])
    return chat_history

In [33]:
# Create Gradio Blocks interface
with gr.Blocks() as demo:
    title = "# Sales Assistant Chatbot"
    gr.Markdown(title)
    category = gr.State()

    with gr.Row():
        with gr.Column(scale=1):
            commodity_input = gr.Textbox(label="Input Drug Name")
            classify_button = gr.Button("Step1.Classify")
            category_output = gr.Textbox(label="Category", interactive=False)
            suggestions_button = gr.Button("Step2.Generate Analysis")
            known_facts_output = gr.Textbox(label="Known Facts", interactive=False)
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(label="Pharmacy Store Chatbot")
            chatbot_input = gr.Textbox(label="Your Message")
            send_button = gr.Button("Continue Chat")
            # html_button = gr.Button("Open US State Profit Map")
            # with gr.Row():
            #     # 2 images in a row
            #     image1 = gr.Image(value=None, label="Top 10 States Distribution")
            #     image2 = gr.Image(value=None, label="Top 10 Cities Distribution")
            # with gr.Row():
            #     image3 = gr.Image(value=None, label="Seasonal Analysis")
            # with gr.Row():
            #     image4 = gr.HTML(label="Discount VS Sales and Profit")
            #     image5 = gr.HTML(label="Sales Over Time")
            # with gr.Row():
            #     image6 = gr.HTML(label="Ship Mode Distribution")
            #     image7 = gr.HTML(label="Unit Price Profit Discount By Segment")



    classify_button.click(fn=classify_commodity, inputs=commodity_input, outputs=category_output)
    # suggestions_button.click(fn=generate_suggestions, inputs=[chatbot, commodity_input, category_output], outputs=[chatbot, known_facts_output])
    suggestions_button.click(
        fn=lambda chatbot, commodity_name, category: 
            generate_suggestions(chatbot, commodity_name, category,
            # read_svgs(category, "discount_vs_avg_sales_volumes_and_profit"),
            # read_svgs(category, "sales_over_time"),
            # read_svgs(category, "ship_mode_distribution"),
            # read_svgs(category, "unit_price_profit_discount_by_segment"),
            # read_html(category)
        ),
        inputs=[chatbot, commodity_input, category_output],
        outputs=[chatbot, known_facts_output],
    )
    send_button.click(fn=send_message, inputs=[chatbot, chatbot_input], outputs=chatbot)
    # html_button.click(fn=open_html, inputs=category_output)

demo.launch(share=True)

C:\Users\10521\.conda\envs\Pytorch\Lib\site-packages\gradio\components\chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://0cccb5be0719991e36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
